# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
main_df = pd.read_csv("output_data.csv", index_col=0)
main_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Punta Arenas,CL,-53.15,-70.92,1.00,86,75,8.20,09/29/2020
1,Puerto Ayora,EC,-0.74,-90.35,20.00,89,93,1.79,09/29/2020
2,Grand Gaube,MU,-20.01,57.66,21.67,84,95,1.34,09/29/2020
3,Aykhal,RU,66.00,111.50,-4.00,79,90,3.00,09/29/2020
4,Ushuaia,AR,-54.80,-68.30,1.00,80,75,9.80,09/29/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = main_df[["Lat", "Long"]]
weights = main_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False))
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
    
hotel_df = main_df.loc[(main_df["Max Temp"] > 25) & (main_df["Max Temp"] < 30) & (main_df["Wind Speed"] < 15) & (main_df["Cloudiness"] < 60)]             
hotel_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
7,Black River,JM,18.03,-77.85,27.00,83,40,0.50,09/29/2020
19,Kapaa,US,22.08,-159.32,29.00,66,1,3.10,09/29/2020
24,Road Town,VG,18.42,-64.62,28.33,88,1,4.92,09/29/2020
39,Marawi,PH,8.00,124.29,28.89,75,53,3.13,09/29/2020
40,Butaritari,KI,3.07,172.79,28.13,75,20,5.79,09/29/2020
42,Tessalit,ML,20.20,1.01,29.36,14,0,2.16,09/29/2020
48,Fomboni,KM,-12.28,43.74,25.20,79,3,6.14,09/29/2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
Hotel_name = []
exception = 1
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    params["location"] = f"{lat},{long}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel = response["results"][0]["name"]
        Hotel_name.append(hotel)
    except:
        print(f"This is the #{exception} exception")
        Hotel_name.append("none")
        exception = exception + 1
        pass


This is the #1 exception
This is the #2 exception


In [7]:
hotel_df.insert(8, "Hotel Name", Hotel_name)
hotel_data = hotel_df.loc[hotel_df["Hotel Name"]!="none"]
new_hotel_data = hotel_data.drop(["Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"], axis=1)
new_hotel_data

,City,Country,Lat,Long,Hotel Name
7,Black River,JM,18.03,-77.85,Idlers' Rest Beach Hotel
19,Kapaa,US,22.08,-159.32,Hilton Garden Inn Kauai Wailua Bay
24,Road Town,VG,18.42,-64.62,Wyndham Tortola BVI Lambert Beach Resort
39,Marawi,PH,8.00,124.29,Marawi Resort Hotel
48,Fomboni,KM,-12.28,43.74,Le Relais de Singani hôtel


In [8]:
a = new_hotel_data["Lat"].tolist()
b = new_hotel_data["Long"].tolist()
x = zip(a, b)
coordinates = list(x)
coordinates

[(18.03, -77.85),
 (22.08, -159.32),
 (18.42, -64.62),
 (8.0, 124.29),
 (-12.28, 43.74)]

In [32]:
names = []
# NOTE: Do not change any of the code in this cell
for index, row in hotel_data.iterrows():
    names.append(f'<b>City:</b><br>{row["Hotel Name"]"]}<br><br><b>Country:</b><br>{row["City"]}<br><br><b>Hotel:</b><br>{row["Country"]}')
    
names
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
#locations = hotel_df[["Lat", "Long"]]

SyntaxError: f-string: unterminated string (<ipython-input-32-7ab536a35625>, line 4)

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates,info_box_content=names)
fig.add_layer(markers)
fig
# Display figure


IndexError: list index out of range